In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding,BertTokenizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
BASE_MODEL = "bert-base-uncased"
LEARNING_RATE = 0.001
MAX_LENGTH = 256
BATCH_SIZE = 32
EPOCHS = 10


# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer =BertTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for param in model.bert.parameters():
    param.requires_grad=False

In [ ]:
import pandas as pd
import os
import torch
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from tqdm import tqdm
# from datasets import Dataset
import warnings
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_excel('/content/drive/MyDrive/INTERIIT/behaviour_simulation_train.xlsx', usecols=[1,2,3,4,5,6])
df.head()

,date,likes,content,username,media,inferred company
0,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>",TimHortonsPH,"[Photo(previewUrl='https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=large')]",tim hortons
1,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an HOUR <hyperlink> <hyperlink>,IndyMusic,"[Photo(previewUrl='https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=large')]",independent
2,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on military drone tech exports to Turkey <hyperlink> <hyperlink>,CBCCanada,"[Photo(previewUrl='https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?format=jpg&name=large')]",cbc
3,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, Italy redoubled its efforts, and they're now paying off. My latest. 👇🏼👇🏼\n<hyperlink> <hyperlink>",MKWilliamsRome,"[Photo(previewUrl='https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?format=jpg&name=large')]",williams
4,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>! She was presented a Beyond the Call Award for her leadership and planning last week’s Cultural Diversity Night! #excellenceisworththeeffort <hyperlink>,BGISD,"[Photo(previewUrl='https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?format=jpg&name=large')]",independent


In [ ]:
pd.set_option('display.max_colwidth', None)
df = df.drop(df.index[14518:14545])
# df.reset_index(drop=True)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
captions = pd.read_csv('/content/drive/MyDrive/INTERIIT/captions-cont.csv')
# captions2=pd.read_csv('./captions-cont.csv')
# combined_captions = pd.concat([captions, captions2], ignore_index=True)
# Replace empty or missing values with 'No information'
captions.fillna('No information', inplace=True)

In [ ]:
captions

,Captions
0,Picture of Tim Hortons
1,Picture of the rapper is in the studio
2,Picture of A soldier in afghanistan
3,"Picture of The beach in the town of sicily, italy"
4,Picture of a student receiving a certificate from a teacher
...,...
27080,a pug
27081,A man on a bridge
27082,a woman holding a piece of food and a set of plastic rockets
27083,"The picture shows the president of nigeria, mr umaru umaru"


In [ ]:
df_new = pd.concat([df, captions], axis=1)

In [ ]:
df_new.dropna(inplace=True)

In [ ]:
df_new['text'] = (
    "Input: \n"
    "The image of a tweet has the following information:\n" +
    df_new['Captions'] + "\n" +
    "...\n"+
    "It was posted by " + df_new['username'] + " on Twitter with the content '" +
    df_new['content'] + "' on " + df_new['date'] + ". The inferred company based on the tweet was " +
    df_new['inferred company'] + " company.  Predict the amount of likes that the tweet got anywhere from \"0\" to \"2000\", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
)


In [ ]:
columns_to_drop = ['date', 'username', 'inferred company', 'Captions','media','content']
df_new.drop(columns=columns_to_drop, inplace=True)
df_new

,likes,text
0,1,"Input: \nThe image of a tweet has the following information:\nPicture of Tim Hortons\n...\nIt was posted by TimHortonsPH on Twitter with the content 'Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>' on 2020-12-12 00:47:00. The inferred company based on the tweet was tim hortons company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
1,2750,"Input: \nThe image of a tweet has the following information:\nPicture of the rapper is in the studio\n...\nIt was posted by IndyMusic on Twitter with the content 'Watch rapper <mention> freestyle for over an HOUR <hyperlink> <hyperlink>' on 2018-06-30 10:04:20. The inferred company based on the tweet was independent company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
2,57,"Input: \nThe image of a tweet has the following information:\nPicture of A soldier in afghanistan\n...\nIt was posted by CBCCanada on Twitter with the content 'Canadian Armenian community demands ban on military drone tech exports to Turkey <hyperlink> <hyperlink>' on 2020-09-29 19:47:28. The inferred company based on the tweet was cbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
3,152,"Input: \nThe image of a tweet has the following information:\nPicture of The beach in the town of sicily, italy\n...\nIt was posted by MKWilliamsRome on Twitter with the content '1st in Europe to be devastated by COVID-19, Italy redoubled its efforts, and they're now paying off. My latest. 👇🏼👇🏼\n<hyperlink> <hyperlink>' on 2020-10-01 11:40:09. The inferred company based on the tweet was williams company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
4,41,"Input: \nThe image of a tweet has the following information:\nPicture of a student receiving a certificate from a teacher\n...\nIt was posted by BGISD on Twitter with the content 'Congratulations to Pauletha Butts of <mention>! She was presented a Beyond the Call Award for her leadership and planning last week’s Cultural Diversity Night! #excellenceisworththeeffort <hyperlink>' on 2018-10-19 14:30:46. The inferred company based on the tweet was independent company. Predict the amount of likes that the tweet got anywher

In [ ]:
df_new.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_likes = df_new['likes']

# Reshape the column to a 2D array (required by StandardScaler)
scaled_likes = scaled_likes.values.reshape(-1, 1)

# Instantiate the StandardScaler
scaler = StandardScaler()

# Fit the scaler to your data and transform the column
scaled_column = scaler.fit_transform(scaled_likes)

# Include the scaled values back into your DataFrame
df_new['scaled_likes'] = scaled_column

In [ ]:
selected_columns = ["text","scaled_likes"]
df_scaled=df_new[selected_columns].copy()

In [ ]:
df_scaled

,text,scaled_likes
0,"Input: \nThe image of a tweet has the following information:\nPicture of Tim Hortons\n...\nIt was posted by TimHortonsPH on Twitter with the content 'Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>' on 2020-12-12 00:47:00. The inferred company based on the tweet was tim hortons company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes.",-0.183195
1,"Input: \nThe image of a tweet has the following information:\nPicture of the rapper is in the studio\n...\nIt was posted by IndyMusic on Twitter with the content 'Watch rapper <mention> freestyle for over an HOUR <hyperlink> <hyperlink>' on 2018-06-30 10:04:20. The inferred company based on the tweet was independent company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes.",0.493020
2,"Input: \nThe image of a tweet has the following information:\nPicture of A soldier in afghanistan\n...\nIt was posted by CBCCanada on Twitter with the content 'Canadian Armenian community demands ban on military drone tech exports to Turkey <hyperlink> <hyperlink>' on 2020-09-29 19:47:28. The inferred company based on the tweet was cbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes.",-0.169420
3,"Input: \nThe image of a tweet has the following information:\nPicture of The beach in the town of sicily, italy\n...\nIt was posted by MKWilliamsRome on Twitter with the content '1st in Europe to be devastated by COVID-19, Italy redoubled its efforts, and they're now paying off. My latest. 👇🏼👇🏼\n<hyperlink> <hyperlink>' on 2020-10-01 11:40:09. The inferred company based on the tweet was williams company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes.",-0.146051
4,"Input: \nThe image of a tweet has the following information:\nPicture of a student receiving a certificate from a teacher\n...\nIt was posted by BGISD on Twitter with the content 'Congratulations to Pauletha Butts of <mention>! She was presented a Beyond the Call Award for her leadership and planning last week’s Cultural Diversity Night! #excellenceisworththeeffort <hyperlink>' on 2018-10-19 14:30:46. The inferred company based on the tweet was independent company. Predict the amount of lik

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

train_ratio = 0.7  # 70% train
val_ratio = 0.15   # 15% validation
test_ratio = 0.15   # 15% test

train_val, test = train_test_split(df_scaled, test_size=test_ratio, random_state=42)
train, val = train_test_split(train_val, test_size=val_ratio/(1-test_ratio), random_state=42)

# Converting the split data into Datasets
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)
test_dataset = Dataset.from_pandas(test)

In [ ]:
ds = {"train": train_dataset, "validation": val_dataset, "test": test_dataset}

def preprocess_function(examples):
    label = examples["scaled_likes"]
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256,add_special_tokens=True,)

    # Change this to real number
    examples["label"] = float(label)
    return examples

for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=["text", "scaled_likes", "__index_level_0__"])


Map:   0%|          | 0/18959 [00:00<?, ? examples/s]

Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    # labels = labels.reshape(-1)
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(y_true=scaler.inverse_transform(labels), y_pred=scaler.inverse_transform(logits))
    # mae = mean_absolute_error(labels, logits)
    rmse = np.sqrt(mse)
    # r2 = r2_score(labels, logits)
    # single_squared_errors = ((logits - labels).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    # accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
    max_logits = np.max(logits)  # Compute max value from logits

    # Log only scalar values for TensorBoard
    # scalar_metrics = {"mse": mse, "rmse": rmse, "max_logits": max_logits}

    # Return the scalar metrics along with the logits and labels for further analysis
    # return scalar_metrics, {'logits': logits, 'labels': labels}
    return {"mse": mse, "rmse": rmse,"labelshape":labels.shape, "logits":logits.shape}

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bert_regression",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=400,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="rmse",
    greater_is_better=False,
    lr_scheduler_type="linear",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

In [ ]:
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        print(outputs.get("logits"))
        print("......................")
        print(labels)
        logits = outputs.get("logits")
        loss = torch.nn.functional.mse_loss(logits.squeeze(), labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics_for_regression,
)

In [ ]:
trainer.train()

tensor([[ 0.3464],
        [ 0.1756],
        [ 0.4443],
        [ 0.2091],
        [ 0.2427],
        [ 0.0226],
        [ 0.4019],
        [ 0.3789],
        [ 0.2701],
        [ 0.1533],
        [ 0.0637],
        [ 0.0185],
        [ 0.1565],
        [ 0.2488],
        [ 0.2853],
        [ 0.3138],
        [ 0.3549],
        [ 0.1808],
        [ 0.3756],
        [ 0.3421],
        [ 0.2338],
        [ 0.1846],
        [ 0.0639],
        [ 0.2781],
        [ 0.4294],
        [ 0.2775],
        [ 0.4027],
        [ 0.1462],
        [ 0.2896],
        [ 0.2852],
        [ 0.3578],
        [ 0.2735],
        [ 0.0499],
        [ 0.1461],
        [ 0.0657],
        [ 0.4397],
        [ 0.1356],
        [ 0.2925],
        [ 0.0151],
        [ 0.4932],
        [ 0.2613],
        [ 0.4260],
        [ 0.1944],
        [ 0.2829],
        [ 0.3403],
        [-0.0451],
        [ 0.2198],
        [ 0.0768],
        [ 0.3280],
        [ 0.0633],
        [ 0.2876],
        [ 0.1269],
        [ 0.

Epoch,Training Loss,Validation Loss,Mse,Rmse,Labelshape,Logits
1,No log,1.775221,29338120.000000,5416.467285,"(4063, 1)","(4063, 1)"
2,No log,1.767168,29205030.000000,5404.167969,"(4063, 1)","(4063, 1)"
3,No log,1.766239,29189674.000000,5402.747070,"(4063, 1)","(4063, 1)"
4,No log,1.763645,29146808.000000,5398.778320,"(4063, 1)","(4063, 1)"
5,No log,1.762670,29130692.000000,5397.285645,"(4063, 1)","(4063, 1)"
6,No log,1.773759,29313950.000000,5414.235840,"(4063, 1)","(4063, 1)"
7,No log,1.764889,29167374.000000,5400.682617,"(4063, 1)","(4063, 1)"
8,No log,1.762750,29132016.000000,5397.408203,"(4063, 1)","(4063, 1)"
9,No log,1.763020,29136472.000000,5397.820801,"(4063, 1)","(4063, 1)"
10,No log,1.762117,29121546.000000,5396.438477,"(4063, 1)","(4063, 1)"


Streaming output truncated to the last 5000 lines.
        -1.4900e-01,  3.0813e-02, -9.1937e-04, -1.8344e-01, -1.5540e-01,
        -1.8319e-01, -1.7754e-01, -1.8172e-01, -1.8319e-01, -1.8319e-01,
         2.9353e-01, -1.3769e-01, -1.3941e-01, -1.3720e-01, -1.8270e-01,
        -1.8319e-01, -9.3902e-02, -1.6302e-01, -1.2637e-01, -1.7139e-01,
         4.1873e-01, -1.7188e-01, -1.8295e-01, -4.0031e-02,  1.4470e-01,
        -1.1407e-01, -1.8344e-01, -1.6721e-01,  2.8073e+00,  1.1445e-01,
         1.4962e-01,  9.0204e+00, -1.5072e-01, -1.0965e-01, -1.6917e-01,
        -1.4285e-01, -1.8344e-01, -1.6942e-01, -1.8098e-01, -1.8147e-01,
        -1.7065e-01, -1.6475e-01, -1.8319e-01, -1.8270e-01, -6.6598e-02,
         3.8795e+00, -1.8319e-01,  1.7077e+00,  1.9882e-01, -9.5289e-03,
        -1.8319e-01,  1.9251e-02, -1.8270e-01, -1.3646e-01, -1.8270e-01,
         2.9008e-01, -1.8221e-01,  6.0986e-01, -1.8172e-01, -1.7975e-01,
        -1.6967e-01, -1.7262e-01, -1.6548e-01, -1.8270e-01,  3.8537e+00,


Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/labelshape" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/logits" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Streaming output truncated to the last 5000 lines.
        -1.8246e-01, -1.8344e-01, -1.7262e-01, -1.6376e-01, -1.8295e-01,
        -1.8344e-01, -1.8147e-01, -1.5712e-01, -1.6868e-01, -1.8344e-01,
         2.1235e-01, -1.4458e-01,  6.0331e-02, -1.8344e-01, -1.3055e-01,
        -1.5220e-01,  3.4297e-01, -1.5392e-01,  9.4736e-01,  1.9366e-01,
         4.3604e-02, -1.8344e-01, -1.8344e-01, -1.8344e-01, -1.8344e-01,
        -1.1186e-01, -1.6745e-01, -3.9047e-02, -8.7506e-02,  7.3728e-01,
        -5.6020e-02, -1.6598e-01, -1.8319e-01, -1.6376e-01, -1.8344e-01,
        -4.3632e-03,  1.1568e-01, -9.2829e-03,  1.9989e-02, -1.6696e-01,
         1.3454e+00, -1.6106e-01,  2.7188e-01, -5.9218e-02, -1.8295e-01,
        -1.8024e-01, -1.8344e-01, -1.8172e-01, -1.8074e-01, -4.4213e-02,
        -1.2317e-01,  4.9425e-01, -1.1629e-01, -1.5614e-01, -1.5392e-01,
         7.6065e-01, -1.1383e-01, -1.8344e-01, -1.8295e-01, -1.8344e-01,
        -1.8270e-01, -1.8221e-01, -1.6573e-01,  3.4002e-01, -1.7336e-01,


...

Streaming output truncated to the last 5000 lines.
         9.8459e-02, -1.8295e-01, -1.2071e-01, -1.7336e-01, -1.8221e-01,
         3.3953e-01, -1.1776e-01, -1.8344e-01, -6.5368e-02, -1.8270e-01,
        -1.7754e-01, -1.7163e-01, -1.8344e-01, -1.8344e-01, -1.4212e-01,
        -1.8344e-01, -1.8344e-01, -3.7817e-02, -1.7852e-01, -1.5884e-01,
        -1.4138e-01, -1.8172e-01, -1.6302e-01,  2.6538e+00, -1.8246e-01,
        -1.4679e-01, -1.3769e-01, -1.8319e-01, -1.6327e-01, -4.7165e-02,
        -1.6745e-01, -1.7090e-01, -1.8246e-01, -6.4630e-02, -1.4384e-01,
        -1.8270e-01, -1.4826e-01,  2.9057e-01, -1.8270e-01, -1.2834e-01,
        -1.8344e-01, -1.8270e-01, -5.4052e-02, -1.5491e-01, -1.8344e-01,
        -1.8074e-01, -1.8344e-01, -1.8196e-01, -1.5909e-01, -1.2342e-01,
        -1.8024e-01, -1.0866e-01, -1.2982e-01, -5.0609e-02, -1.8344e-01,
        -1.7065e-01, -6.9549e-02, -1.4335e-01, -1.5491e-01, -1.7311e-01,
        -2.0598e-02, -1.7336e-01, -1.5663e-01, -1.8123e-01, -1.6524e-01,


Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/labelshape" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/logits" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Streaming output truncated to the last 5000 lines.
        -1.5860e-01, -1.8344e-01, -1.8344e-01, -7.3150e-03, -1.5860e-01,
        -1.6868e-01, -1.8319e-01, -1.8147e-01, -1.1654e-03, -1.8319e-01,
        -1.8344e-01, -1.5909e-01, -1.0054e-01,  1.1961e-01, -5.5036e-02,
        -1.8344e-01, -1.8344e-01, -1.6844e-01, -1.5417e-01, -1.8270e-01,
        -1.8295e-01, -1.8344e-01, -1.7975e-01, -1.8319e-01,  2.6745e-01,
        -1.0350e-01, -1.6548e-01,  1.6119e-01, -1.8319e-01, -1.4876e-01,
         3.9437e+00, -1.6352e-01,  3.7470e-01, -1.5122e-01, -1.8246e-01,
        -1.8344e-01,  5.1565e-01, -1.8246e-01, -1.8344e-01, -8.3571e-02,
        -1.5687e-01, -1.7951e-01, -1.5417e-01,  1.5725e-01, -1.1251e-02,
        -1.4949e-01, -1.6991e-01, -1.7040e-01, -1.4949e-01, -1.6302e-01,
         4.3275e-01, -1.7016e-01, -1.8319e-01, -1.8344e-01, -1.7213e-01,
        -1.4925e-01, -1.8172e-01, -1.8295e-01, -1.7016e-01, -1.7286e-01,
        -1.6819e-01, -6.4384e-02, -1.4581e-01, -8.9474e-02, -1.4285e-01,


Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/labelshape" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/logits" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=240, training_loss=0.9607236226399739, metrics={'train_runtime': 3731.0672, 'train_samples_per_second': 50.814, 'train_steps_per_second': 0.064, 'total_flos': 2.494138855217664e+16, 'train_loss': 0.9607236226399739, 'epoch': 10.0})

In [ ]:
trainer.eval_dataset=ds["test"]
trainer.evaluate()

tensor([[ 1.6304e-03],
        [-5.3467e-03],
        [ 3.8185e-02],
        [ 4.0045e-02],
        [ 2.4222e-02],
        [-1.6509e-02],
        [ 1.7229e-02],
        [ 2.4243e-02],
        [ 1.0291e-01],
        [ 3.4244e-02],
        [-5.8528e-02],
        [-3.4765e-02],
        [ 1.1033e-01],
        [ 2.6122e-02],
        [-1.2355e-02],
        [-3.7960e-02],
        [ 4.4141e-02],
        [ 6.3324e-03],
        [-3.3225e-02],
        [-1.7912e-02],
        [ 4.0312e-03],
        [ 7.1897e-03],
        [-3.6932e-03],
        [ 1.5935e-02],
        [ 6.9530e-03],
        [ 4.8885e-02],
        [-5.1133e-02],
        [ 4.9105e-03],
        [ 8.5055e-03],
        [ 2.1046e-02],
        [-2.2841e-02],
        [ 3.8623e-02],
        [ 3.1256e-02],
        [ 3.2606e-02],
        [ 2.9283e-02],
        [ 7.0299e-02],
        [ 1.2356e-02],
        [-1.3895e-02],
        [-2.1643e-02],
        [ 3.4793e-02],
        [-2.9297e-03],
        [ 1.6122e-02],
        [-2.8164e-02],
        [ 5

tensor([[ 2.4200e-02],
        [-1.5874e-02],
        [ 2.7382e-03],
        [ 6.7370e-03],
        [-1.8841e-02],
        [ 2.4908e-03],
        [-1.9925e-02],
        [ 2.2642e-02],
        [ 1.1272e-01],
        [ 5.2913e-02],
        [ 4.0248e-02],
        [ 1.5858e-02],
        [-3.4661e-02],
        [ 1.4181e-02],
        [-2.0730e-02],
        [-3.0484e-02],
        [-1.5382e-02],
        [-4.4284e-02],
        [-3.7986e-02],
        [ 2.7569e-02],
        [ 2.4254e-02],
        [-1.7290e-02],
        [ 5.0404e-02],
        [ 2.0306e-02],
        [ 2.7377e-02],
        [ 4.6656e-02],
        [-2.0920e-02],
        [ 3.9312e-02],
        [ 2.4135e-02],
        [-1.7909e-03],
        [ 1.5156e-02],
        [ 4.8895e-02],
        [-3.7336e-02],
        [ 2.9164e-02],
        [-2.7504e-02],
        [ 3.7714e-03],
        [-2.5873e-02],
        [-3.0704e-02],
        [ 1.3058e-02],
        [ 4.6604e-02],
        [-4.1792e-03],
        [-3.0028e-02],
        [ 1.4384e-02],
        [-2

Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/labelshape" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(4063, 1)" of type <class 'tuple'> for key "eval/logits" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.48671793937683105,
 'eval_mse': 8043724.5,
 'eval_rmse': 2836.14599609375,
 'eval_labelshape': (4063, 1),
 'eval_logits': (4063, 1),
 'eval_runtime': 64.4529,
 'eval_samples_per_second': 63.038,
 'eval_steps_per_second': 0.171,
 'epoch': 10.0}

In [ ]:
model_directory = "/content/drive/MyDrive/INTERIIT/bert-trained1"  # Directory to save the model

# Save the trained model and tokenizer
model.save_pretrained(model_directory)  # Save the model weights and configuration
tokenizer.save_pretrained(model_directory)

('/content/drive/MyDrive/INTERIIT/bert-trained1/tokenizer_config.json',
 '/content/drive/MyDrive/INTERIIT/bert-trained1/special_tokens_map.json',
 '/content/drive/MyDrive/INTERIIT/bert-trained1/vocab.txt',
 '/content/drive/MyDrive/INTERIIT/bert-trained1/added_tokens.json')

# Prediction on data


In [ ]:
df_result1=pd.read_excel('./behaviour_simulation_test_company.xlsx')
df_result1

,id,date,content,username,media,inferred company
0,1,2019-01-08 18:03:10,the Heathrow drone right now <hyperlink>,bbcthree,"[Photo(previewUrl='https://pbs.twimg.com/media/DwaFKixW0AIjsun?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/DwaFKixW0AIjsun?format=jpg&name=large')]",bbc
1,2,2018-01-29 10:51:17,A deal at £60m?\n\nThe latest on Pierre-Emerick Aubameyang with Arsenal closing in on a transfer: <hyperlink> <hyperlink>,BBCSport,"[Photo(previewUrl='https://pbs.twimg.com/media/DUs_TqAXkAAUzvB?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/DUs_TqAXkAAUzvB?format=jpg&name=large')]",bbc
2,3,2019-09-05 07:25:14,"""Dealing with Boris Johnson is like someone dealing with a two or three year old having a tantrum,"" says shadow chancellor John McDonnell, ""this is about putting the country before party interest""\n\n<hyperlink> #Brexit <hyperlink>",BBCPolitics,"[Video(thumbnailUrl='https://pbs.twimg.com/media/EDrwxl9UUAEDktR.jpg', variants=[VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1169511449984557058/vid/1280x720/i0shI8-WQf8w5Mrs.mp4?tag=13', bitrate=2176000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1169511449984557058/vid/480x270/MVlE3Zf9v2aeyenP.mp4?tag=13', bitrate=288000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1169511449984557058/vid/640x360/n91aUVr7pS5wsPor.mp4?tag=13', bitrate=832000), VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/amplify_video/1169511449984557058/pl/__4MfLL7xiYeoKz_.m3u8?tag=13', bitrate=None)], duration=140.144, views=46572)]",bbc
3,4,2018-03-28 18:15:00,Doesn't unlimited food &amp; drinks all night at #TellMeYouLoveMeTour sound pretty good?\n\nUpgrade your 🎟️ to <mention> and it's yours! <hyperlink> <hyperlink>,BrdgstoneArena,"[Photo(previewUrl='https://pbs.twimg.com/media/DZYxnDVXkAILX_H?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/DZYxnDVXkAILX_H?format=jpg&name=large')]",bridgestone
4,5,2019-09-11 21:15:44,Apple Watch Series 5.\n\nYou’ve never seen a watch like this. This watch tells time. (Among other things.) <hyperlink> <hyperlink>,BestBuy,"[Video(thumbnailUrl='https://pbs.twimg.com/ext_tw_video_thumb/1171895097207808001/pu/img/AX25Xwg1LQcWPyyD.jpg', variants=[VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/ext_tw_video/1171895097207808001/pu/pl/GuC2Z1vzGmwNq4xs.m3u8?tag=10', bitrate=None), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1171895097207808001/pu/vid/640x360/l67vYVgl-IwBrQYT.mp4?tag=10', bitrate=832000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1171895097207808001/pu/vid/1280x720/duJQtKc25IB2IBrB.mp4?tag=10', bitrate=2176000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1171895097207808001/pu/vid/480x270/bgcl8y375WwY_0c1.mp4?tag=10', bitrate=256000)], duration=13.7, views=5706)]",best buy
...,...,...,...,...,...,...
9995,9996,2020-01-25 01:45:07,"""So what do all these defenses mean? What they mean is: 'Under Article II, the President can do whatever he wants.' ... They want us to believe the President can do whatever he wants under Article II, and there is nothing you or the House can do about it."" - <mention> <hyperlink>",HouseIntelDems,"[Video(thumbnailUrl='https://pbs.twimg.com/media/EPF0EOxX4AEx2-M.jpg', variants=[VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1220884158165241856/vid/1280x720/L6HAzq23M_9H0pLr.mp4?tag=13', bitrate=2176000), VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/amplify_video/1220884158165241856/pl/B_7GLr6TJmYO9Gyo.m3u8?tag=13', bitrate=None), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1220884158165241856/vid/640x360/QX8zJOyZdFl7IK6q.mp4?tag=13', bitrate=832000), VideoVariant(contentType='video/mp4', url='https://video.twimg

In [ ]:
df_result1 = df_result1.reset_index(drop=True)

In [ ]:
df_result1.dropna(inplace=True)

In [ ]:
df_result1['text'] = (
    "Input: \n"
    "Tweet was posted by " + df_result1['username'] + " on Twitter with the content '" +
    df_result1['content'] + "' on " + df_result1['date'] + ". The inferred company based on the tweet was " +
    df_result1['inferred company'] + " company.  Predict the amount of likes that the tweet got anywhere from \"0\" to \"2000\", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
)
df_result1

,id,date,content,username,media,inferred company,text
0,1,2019-01-08 18:03:10,the Heathrow drone right now <hyperlink>,bbcthree,"[Photo(previewUrl='https://pbs.twimg.com/media/DwaFKixW0AIjsun?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/DwaFKixW0AIjsun?format=jpg&name=large')]",bbc,"Input: \nTweet was posted by bbcthree on Twitter with the content 'the Heathrow drone right now <hyperlink>' on 2019-01-08 18:03:10. The inferred company based on the tweet was bbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
1,2,2018-01-29 10:51:17,A deal at £60m?\n\nThe latest on Pierre-Emerick Aubameyang with Arsenal closing in on a transfer: <hyperlink> <hyperlink>,BBCSport,"[Photo(previewUrl='https://pbs.twimg.com/media/DUs_TqAXkAAUzvB?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/DUs_TqAXkAAUzvB?format=jpg&name=large')]",bbc,"Input: \nTweet was posted by BBCSport on Twitter with the content 'A deal at £60m?\n\nThe latest on Pierre-Emerick Aubameyang with Arsenal closing in on a transfer: <hyperlink> <hyperlink>' on 2018-01-29 10:51:17. The inferred company based on the tweet was bbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
2,3,2019-09-05 07:25:14,"""Dealing with Boris Johnson is like someone dealing with a two or three year old having a tantrum,"" says shadow chancellor John McDonnell, ""this is about putting the country before party interest""\n\n<hyperlink> #Brexit <hyperlink>",BBCPolitics,"[Video(thumbnailUrl='https://pbs.twimg.com/media/EDrwxl9UUAEDktR.jpg', variants=[VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1169511449984557058/vid/1280x720/i0shI8-WQf8w5Mrs.mp4?tag=13', bitrate=2176000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1169511449984557058/vid/480x270/MVlE3Zf9v2aeyenP.mp4?tag=13', bitrate=288000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1169511449984557058/vid/640x360/n91aUVr7pS5wsPor.mp4?tag=13', bitrate=832000), VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/amplify_video/1169511449984557058/pl/__4MfLL7xiYeoKz_.m3u8?tag=13', bitrate=None)], duration=140.144, views=46572)]",bbc,"Input: \nTweet was posted by BBCPolitics on Twitter with the content '""Dealing with Boris Johnson is like someone dealing with a two or three year old having a tantrum,"" says shadow chancellor John McDonnell, ""this is about putting the country before party interest""\n\n<hyperlink> #Brexit <hyperlink>' on 2019-09-05 07:25:14. The inferred company based on the tweet was bbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
3,4,2018-03-28 18:15:00,Doesn't unlimit

In [ ]:
columns_to_drop = ['id', 'username', 'inferred company','media','content','date']
df_result1.drop(columns=columns_to_drop, inplace=True)
df_result1

,text
0,"Input: \nTweet was posted by bbcthree on Twitter with the content 'the Heathrow drone right now <hyperlink>' on 2019-01-08 18:03:10. The inferred company based on the tweet was bbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
1,"Input: \nTweet was posted by BBCSport on Twitter with the content 'A deal at £60m?\n\nThe latest on Pierre-Emerick Aubameyang with Arsenal closing in on a transfer: <hyperlink> <hyperlink>' on 2018-01-29 10:51:17. The inferred company based on the tweet was bbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
2,"Input: \nTweet was posted by BBCPolitics on Twitter with the content '""Dealing with Boris Johnson is like someone dealing with a two or three year old having a tantrum,"" says shadow chancellor John McDonnell, ""this is about putting the country before party interest""\n\n<hyperlink> #Brexit <hyperlink>' on 2019-09-05 07:25:14. The inferred company based on the tweet was bbc company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
3,"Input: \nTweet was posted by BrdgstoneArena on Twitter with the content 'Doesn't unlimited food &amp; drinks all night at #TellMeYouLoveMeTour sound pretty good?\n\nUpgrade your 🎟️ to <mention> and it's yours! <hyperlink> <hyperlink>' on 2018-03-28 18:15:00. The inferred company based on the tweet was bridgestone company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
4,"Input: \nTweet was posted by BestBuy on Twitter with the content 'Apple Watch Series 5.\n\nYou’ve never seen a watch like this. This watch tells time. (Among other things.) <hyperlink> <hyperlink>' on 2019-09-11 21:15:44. The inferred company based on the tweet was best buy company. Predict the amount of likes that the tweet got anywhere from ""0"" to ""2000"", based on the date, content, image description, username, and inferred company. Use historical data and relevant events to enhance the predictive capabilities. For instance, leverage known trends, popular topics, or significant events that occurred on specific dates to enrich understanding and prediction accuracy. It is imperative that you only reply by the number of likes. And also vary the likes."
...,...
9995,"Input: \nTweet was posted by HouseIntelDems on Twitter with the content '""So what do all these de

In [ ]:
result_dataset= Dataset.from_pandas(df_result1)

In [ ]:
import math
import pandas as pd
import torch
# Convert the Dataset to a Pandas DataFrame

nb_batches = math.ceil(len(result_dataset)/200)
y_preds = []

for i in range(nb_batches):
    input_texts = result_dataset[i * 200: (i+1) * 200]["text"]
    # input_labels = result_dataset[i * 200: (i+1) * BATCH_SIZE]["scaled_likes"]
    encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=256, add_special_tokens=True)
    input_ids = torch.tensor(encoded['input_ids']).to('cuda:0') # Convert input_ids to a tensor
    attention_mask = torch.tensor(encoded['attention_mask']).to('cuda:0') # Assuming the tokenizer returns attention_mask

    batch_y_preds = model(input_ids, attention_mask=attention_mask).logits

    # Append predictions for the current batch to y_preds list
    y_preds += batch_y_preds.detach().cpu().tolist()
    # encoded = tokenizer(input_texts, truncation=True, padding="max_length", max_length=256,add_special_tokens=True)
    # y_preds = model(**encoded).logits

# pd.set_option('display.max_rows', 500)
# test_df["likes"] = test_df["scaled_likes"] * (max_likes - min_likes) + min_likes
# df_out = pd.DataFrame([test_dataset["text"], test_df["likes"], y_preds_rescaled], ["Text", "likes", "Prediction"])

In [ ]:
y_preds

[[-0.043104395270347595],
 [-0.024238690733909607],
 [-0.04766654968261719],
 [-0.03810597211122513],
 [0.0171564519405365],
 [-0.04722386598587036],
 [-0.02182023972272873],
 [0.009559824131429195],
 [-0.05182495713233948],
 [-0.04524989426136017],
 [-0.00024450637283734977],
 [-0.031956762075424194],
 [-0.01659846305847168],
 [-0.0327247753739357],
 [-0.030325450003147125],
 [-0.0562157928943634],
 [-0.03480539470911026],
 [-0.03751389682292938],
 [-0.03250254690647125],
 [-0.030636563897132874],
 [-0.04063453525304794],
 [-0.025935694575309753],
 [0.005578845273703337],
 [-0.0066224937327206135],
 [-0.0207316055893898],
 [-0.047529272735118866],
 [-0.036799147725105286],
 [-0.01808808743953705],
 [-0.028858281672000885],
 [-0.008970589376986027],
 [-0.0329003632068634],
 [-0.005785525310784578],
 [-0.015528515912592411],
 [-0.023044660687446594],
 [-0.008370147086679935],
 [-0.0167529433965683],
 [-0.003967583645135164],
 [-0.03366099298000336],
 [-0.008732722140848637],
 [0.0302066

In [ ]:
y_preds_np = np.array(y_preds).reshape(-1, 1)

In [ ]:
y_preds_np

array([[-0.0431044 ],
       [-0.02423869],
       [-0.04766655],
       ...,
       [-0.06509031],
       [-0.03530386],
       [-0.03483088]])

In [ ]:
y_preds_original_scale = scaler.inverse_transform(y_preds_np)

In [ ]:
y_preds_original_scale

array([[570.50629012],
       [647.20055946],
       [551.9598797 ],
       ...,
       [481.12749511],
       [602.2176043 ],
       [604.14041905]])

In [ ]:
y_preds_int = np.round(y_preds_original_scale).astype(int)

In [ ]:
y_preds_int

array([[571],
       [647],
       [552],
       ...,
       [481],
       [602],
       [604]])

In [ ]:
np.savetxt('behavior_test_company_results.csv', y_preds_int, delimiter=',', fmt='%d')

In [ ]:
scaler_params = scaler.get_params()
print(scaler_params)

{'copy': True, 'with_mean': True, 'with_std': True}


In [ ]:
scaler_means = scaler.mean_
scaler_std_devs = np.sqrt(scaler.var_)

# Save the mean and standard deviation values to use in a different code
# Example: Save to a file
np.savetxt('/content/drive/MyDrive/INTERIIT/scaler_means.txt', scaler_means)
np.savetxt('/content/drive/MyDrive/INTERIIT/scaler_std_devs.txt', scaler_std_devs)